In [1]:
import math
import random
import torch
import matplotlib.pyplot as plt
from scipy.spatial import distance
import itertools
import jellyfish
import numpy as np

In [35]:
%run partial_words.py
%run core.py
%run imgCls.py

In [ ]:
def choose_words(nm):
    return random.choices(sbst,k=nm)



In [ ]:
len(choose_words(5000))

In [ ]:
sbst

In [ ]:
spl_Tree = {}

In [ ]:
for p in itertools.permutations([1,2,3,4,5],2):
    print(p)

In [ ]:

def name_path(name):
    return "./net_data/" + name + ".bin"
def save_network(ntwrk,name):
    nt_path = name_path(name)
    torch.save(ntwrk.state_dict(),nt_path)
    return

def load_network(name):
    nt_pth = name_path(name)
    return torch.load(nt_pth)



In [ ]:
def test_words(wrds):
    wrl = WordManage()
    wrl.set_words(wrds,0)
    mdl = build_choose_and_train(wrl,dbg=True,out_cat=5)
    wrds = wrl.all_words()
    cats = get_category(mdl,wrds,dbg=True)
    print("categories... ",cats)
    return

In [ ]:
test_wrds =['food', 'wood']
10 % 20

In [ ]:
test_words(test_wrds)

In [ ]:
im = ImgNet(5)
im.apply(custom_init_weights)
im = best_move(im)
get_category(im,['Chris','Schrumm','Programmer','Test','Nice','Some','Zero','Whatever','Mouse','Other'])

In [ ]:
tensor_info(numpy_to_tensor(np.array([3,4,5])))

In [ ]:
ww = word_batch(['Chris','Schrumm'])

In [ ]:
def display_im(im):
    plt.figure(figsize=(25, 150),dpi=80)
    plt.imshow(im)

In [ ]:
display_im(ww[0][0])

In [ ]:
tt = numpy_to_tensor(np.array([3,4,5]))
tt.to(best_device())
tensor_info(tt)

In [ ]:
word_batch(["chris","schrumm"]).shape

In [147]:
# rtTreeName contains the name of the network to load 
# avWords (instance of WordManage) contains the list of available words.
# wdDic contains the chosen words root network choice
# maxDepth
def train_and_choose(rtTreeName, avWords, wdDic, maxDepth, btch_count):
    '''
    
    '''
    print("train tree..")
    mdl = build_choose_and_train(avWords)
   
    # write the code to save the model..
    save_network(mdl,rtTreeName)

    all_wrds = avWords.all_words()

    #print(type(all_wrds),"      ",type(all_wrds[0]))

    cat_w = get_category(mdl,all_wrds)

    for i,wrd in enumerate(all_wrds):
        #print(wrd,"  ",cat_w[i])
        avWords.set_category(wrd,cat_w[i])
   
    if maxDepth <= 0:
        print("max depth exceeded..")
        return

    for i in range(5):
        sb_tree = rtTreeName + "_" + str(i)
        nw_wrl = avWords.filter_list(i)
        
        for w in nw_wrl.all_words():
            wdDic[w] = sb_tree
    
        print("category ",i," total word count ",len(nw_wrl.all_words()))
    
        # recurse..
        wrds = nw_wrl.all_words()
        if len(wrds) > 1:
            # recurse..
            print("recursing on ",sb_tree," total words.. ", len(wrds))    
            if len(wrds) < 5:
                print("Words..",wrds)
            train_and_choose(sb_tree, nw_wrl, wdDic, maxDepth - 1, btch_count)
    return


def spell_word(word,ntwrk_name,mmp,dr):
     
    
    # load the network
    mdl = best_move(ImgNet(5))
       
    dr = "/" + ntwrk_name + ".bin"
    
    nt = torch.load(dr)
    
    load_data_into(mdl,dr)
    
    
    cats = get_category(mdl,wrds,dbg=True)
    
    print(cats)
    
    word = ""
    if len(word) > 0:
        return word
    
    
    # word = [k for k,v in mmp if v == brtch]

In [ ]:
mpp = load_json("word_tree_mapping.json")
mpp 

NameError: name 'mpp' is not defined

In [6]:
save_json(dcMapping,"word_tree_mapping.json")

In [7]:
data_ky = load_json("word_tree_mapping.json")

In [ ]:
data_ky 

In [29]:
ntwrk_name = "mdl_spelling"
wrd = "marketing"
dr = "./net_data"

mdl = best_move(ImgNet(5))
       
fl = dr + "/" + ntwrk_name + ".bin"
    
nt = torch.load(fl)

mdl.load_state_dict(nt)
mdl.eval()
      
cats = get_category(mdl,[wrd])

In [30]:
cats

array([0])

In [33]:
mmp = data_ky
sb_ntwrk = ntwrk_name + "_" +  str(cats[0])

word = [k for k,v in mmp.items() if v == sb_ntwrk]

In [34]:
sb_ntwrk

'mdl_spelling_0'

In [40]:
# pell_word(wrd,ntwrk_name,mmp,dr, dbg=False):     
spell_word("debate","mdl_spelling",data_ky,"./net_data",dbg=True)

category..  [2]
traverse down..  ./net_data/mdl_spelling_2.bin
category..  [1]
traverse down..  ./net_data/mdl_spelling_2_1.bin
category..  [2]
traverse down..  ./net_data/mdl_spelling_2_1_2.bin
category..  [4]
traverse down..  ./net_data/mdl_spelling_2_1_2_4.bin
category..  [3]
traverse down..  ./net_data/mdl_spelling_2_1_2_4_3.bin
category..  [4]
traverse down..  ./net_data/mdl_spelling_2_1_2_4_3_4.bin
category..  [0]
key  debate  value  mdl_spelling_2_1_2_4_3_4_0


['debate']

In [ ]:
def save_json(sjn,flnm):
    data = json.dumps(sjn)
    with open(flnm,"w") as fl:
        fl.write(data)
    return
def load_json(flnm):
    data = None
    with open(flnm,"r") as fl:
        data = fl.read()
    return json.loads(data)
    

In [5]:
rt_name = "mdl_spelling"
dcMapping = {}
maxDepth = 30
btch_size = 128
wrl = WordManage()
wrl.set_words(sbst,0)
train_and_choose(rt_name,wrl,dcMapping,maxDepth,btch_size)


train tree..
category  0  total word count  131
recursing on  mdl_spelling_0  total words..  131
train tree..
category  0  total word count  40
recursing on  mdl_spelling_0_0  total words..  40
train tree..
category  0  total word count  8
recursing on  mdl_spelling_0_0_0  total words..  8
train tree..
category  0  total word count  1
category  1  total word count  2
recursing on  mdl_spelling_0_0_0_1  total words..  2
Words.. ['exchange', 'evidence']
train tree..
category  0  total word count  1
category  1  total word count  1
category  2  total word count  0
category  3  total word count  0
category  4  total word count  0
category  2  total word count  1
category  3  total word count  2
recursing on  mdl_spelling_0_0_0_3  total words..  2
Words.. ['advantage', 'objective']
train tree..
category  0  total word count  1
category  1  total word count  1
category  2  total word count  0
category  3  total word count  0
category  4  total word count  0
category  4  total word count  2
r

In [ ]:
wrl.word_count()

In [ ]:
def remove_letter(wd):
    if len(wd) < 5:
        return wd
    cs = random.randint(0,len(wd)-1)
    return wd[0:cs] + wd[cs+1:]

def swap_letter(wd):
    if len(wd) < 3:
        return wd
    cs = random.randint(0,len(wd)-2)
    wd = [w for w in wd]
    t = wd[cs]
    wd[cs] = wd[cs+1]
    wd[cs+1] = t
    
    tmp = ""
    for w in wd:
        tmp += w
        
    return tmp

def change_letter(wd):    
    if len(wd) < 4:
        return wd
    
    alpha = ['a','b','c','d','e','f','g', \
        'h','i','j','k','l','m','n','o','p','q','r', \
        's','t','u','v','w','x','y','z']
    cs = random.randint(0,len(wd)-1)

    wd = [w for w in wd]
    
    wd[cs] = alpha[random.randint(0,25)]
    
    tmp = ""
    for w in wd:
        tmp += w
        
    return tmp

def drop_letter(wd):
    if len(wd) < 5:
        return wd
    cs = random.randint(0,len(wd)-2)
    
    wd = [w for w in wd]
    
    wd[cs] = ' '
    
    tmp = ""
    for w in wd:
        tmp += w
        
    return tmp   

def add_letter(wd):
    if len(wd) < 3:
        return wd
    
    #print("add letter")
    cs = random.randint(0,len(wd)-2)
    
    #wd = [w for w in wd]
    
    alpha = ['a','b','c','d','e','f','g', \
        'h','i','j','k','l','m','n','o','p','q','r', \
        's','t','u','v','w','x','y','z']
    
    #ln = len(wd)
    wd = wd[0:cs] + alpha[random.randint(0,25)] + wd[cs:] 
           
    return wd 


def word_mix(wd):
    fc_ar = [remove_letter,swap_letter,change_letter,drop_letter,add_letter]
    
    wd = fc_ar[random.randint(0,4)](wd)
    
    if random.random() < 0.4:
        wd = fc_ar[random.randint(0,4)](wd)
        
    if random.random() < 0.3:
        wd = fc_ar[random.randint(0,4)](wd)
        
    if random.random() < 0.2:
        wd = fc_ar[random.randint(0,4)](wd)
        
    return wd


In [ ]:
nn = "chris"
print(nn[0:3] + nn[3:])

In [ ]:
s = "transportation"
print(len(s))

for i in range(100):    
    nw_s = word_mix(s)
    print(nw_s,"  ",len(nw_s))


In [ ]:
for i in range(30):
    print(random.randint(0,4))

In [ ]:
vl = "nice"
        

In [ ]:
rtTreeName = "mdl_spelling"
avWords = wrl
wdDic = {}

In [ ]:
mdl = build_choose_and_train(avWords)

In [ ]:

# write the code to save the model..
# save_network(mdl,rtTreeName)

all_wrds = avWords.all_words()

#print(type(all_wrds),"      ",type(all_wrds[0]))

cat_w = get_category(mdl,all_wrds)

for i,wrd in enumerate(all_wrds):
    #print(wrd,"  ",cat_w[i])
    avWords.set_category(wrd,cat_w[i])
   

for i in range(5):
    sb_tree = rtTreeName + "_" + str(i)
    nw_wrl = avWords.filter_list(i)
    for w in nw_wrl.all_words():
        wdDic[w] = sb_tree
    
    print("category ",i," total word count ",len(nw_wrl.all_words()))
    # recurse..
    wrds = nw_wrl.all_words()
    if len(wrds) > 0:
        # recurse..
        print("recursing on ",sb_tree)
        # train_and_choose(sb_tree, nw_wrl, wdDic, maxDepth, btch_count

In [ ]:
wdDic

In [ ]:
save_network(mdl,"test")

In [ ]:
mdl = best_move(ImgNet(5))
load_data_into(mdl,"test")
#mdl = load_network("test")

In [ ]:
wrl.set_category('bike',2)
wrl.set_category('cause',2)
wrl.set_category('clerk',2)

In [ ]:
l = wrl.filter_list(2)

In [ ]:
l.choose_words(3)

In [ ]:
for i in range(1000):
    print(random.random())

In [ ]:
# ['tension', 'session']
wd_e = encode_word("food")

In [ ]:
wd_e.shape

In [ ]:
def display_im(im):
    plt.figure(figsize=(25, 150),dpi=80)
    plt.imshow(im)

In [ ]:
display_im(wd_e)

In [ ]:
wd_m = encode_word("wood")

In [ ]:
kys = load_keys()

In [ ]:
mnv,mxv = min(kys['ky'].values()),max(kys['ky'].values())
print(mnv, " ",mxv)
vl = mxv * 0.999
nv = -1 + (2.0 / (mxv - mnv)) * (vl - mnv)
print(nv)

In [ ]:
display_im(wd_m)